In [ ]:
import geopandas as gpd
import os
import warnings
import ast

warnings.filterwarnings("ignore")

# Base directory path
base_dir = r"C:\Users\natda\Desktop\NatDave\Academics\PhD_NU\RESEARCH\Traffic_Stress\Boston"

# Construct file paths
roads_path = os.path.join(base_dir, "street_network.shp")
crossings_path = os.path.join(base_dir, "crossings.shp")

# Load shapefiles
roads = gpd.read_file(roads_path)
crossings = gpd.read_file(crossings_path)

# Convert FRM_LEG, TO_LEG, and CRS_LEG from string to list
crossings['FRM_LEG'] = crossings['FRM_LEG'].apply(ast.literal_eval)
crossings['TO_LEG'] = crossings['TO_LEG'].apply(ast.literal_eval)
crossings['CRS_LEG'] = crossings['CRS_LEG'].apply(ast.literal_eval)

# Create mapping dictionaries from the street network
bike_type_dict = roads.set_index('unique_id')['bike_type'].to_dict()
stoperneu_dict = roads.set_index('unique_id')['StOperNEU'].to_dict()
federalfun_dict = roads.set_index('unique_id')['FEDERALFUN'].to_dict()
adt_infer_dict = roads.set_index('unique_id')['ADT_Infer'].to_dict()
qdirlanes_dict = roads.set_index('unique_id')['qDirLanes'].to_dict()
speed_dict = roads.set_index('unique_id')['SPEED'].to_dict()

# Function to get first element value from mapping dictionary
def get_first_leg_value(leg_list, mapping_dict):
    if leg_list:
        return mapping_dict.get(leg_list[0], "")  # Leave empty if no match
    return ""

# Assign values based on first element in FRM_LEG, TO_LEG, and CRS_LEG
crossings['FR_BK_TYPE'] = crossings['FRM_LEG'].apply(lambda x: get_first_leg_value(x, bike_type_dict))
crossings['TO_BK_TYPE'] = crossings['TO_LEG'].apply(lambda x: get_first_leg_value(x, bike_type_dict))

crossings['FR_StOpNU'] = crossings['FRM_LEG'].apply(lambda x: get_first_leg_value(x, stoperneu_dict))
crossings['TO_StOpNU'] = crossings['TO_LEG'].apply(lambda x: get_first_leg_value(x, stoperneu_dict))

crossings['FR_CLASS'] = crossings['FRM_LEG'].apply(lambda x: get_first_leg_value(x, federalfun_dict))
crossings['TO_CLASS'] = crossings['TO_LEG'].apply(lambda x: get_first_leg_value(x, federalfun_dict))

crossings['CRS_ADT'] = crossings['CRS_LEG'].apply(lambda x: get_first_leg_value(x, adt_infer_dict))
crossings['CRS_LANES'] = crossings['CRS_LEG'].apply(lambda x: get_first_leg_value(x, qdirlanes_dict))

crossings['CRS_SPEED'] = crossings['CRS_LEG'].apply(lambda x: get_first_leg_value(x, speed_dict))

# Create empty columns to be populated later
crossings['CROSS_DIST'] = None              # crossing distance (> 100 ft)
crossings['FREEWAY_RAMP'] = None            # freeway ramp (on/off)

# Save the updated crossings shapefile
crossings.to_file(crossings_path, driver='ESRI Shapefile')

print("Updated crossings.shp with speed limit, bike type, StOperNEU, functional classifications, ADT, and number of lanes.")
print("CROSS_DIST column initialized for future population.")

Updated crossings.shp with speed limit, bike type, StOperNEU, functional classifications, ADT, and number of lanes.
CROSS_DIST column initialized for future population.
